In [486]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [487]:
print("Ello govna") #british for hello#

print("Oi tudo bem?") #portuguese for how are you?#

print("Tuxedo") #american for fancy clothes#

print("Terno") #portuguese for tuxedo#

Ello govna
Oi tudo bem?
Tuxedo
Terno
time: 16 ms (started: 2023-12-09 18:18:29 -05:00)


In [488]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Risk Combined Final Dataset.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,M,140,289,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,49,F,160,180,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,37,M,130,283,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,48,F,138,214,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,54,M,150,195,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


time: 31 ms (started: 2023-12-09 18:18:29 -05:00)


In [489]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})




time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [490]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)

heart_attack.head()

[1 0]
[nan -1.  1.]
[nan -1.  1.]
[nan  0. -1.  1.]
[nan  4.  3.  2.  0.  1.  5.]


,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,1,140,289,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,49,0,160,180,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,37,1,130,283,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,48,0,138,214,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,54,1,150,195,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


time: 16 ms (started: 2023-12-09 18:18:29 -05:00)


In [491]:
mode_columns = ['Stress Level','Medication Use', 'Previous Heart Problems', 'Diet', 'Physical Activity Days Per Week', 'Alcohol Consumption', 'Obesity', 'Smoking', 
                'Family History', 'Diabetes', 'Hemisphere', 'Continent', 'Country']

mean_columns = ['Exercise Hours Per Week', 'Sleep Hours Per Day', 'BMI', 'Sedentary Hours Per Day']

median_columns = ['Heart Rate', 'Income', 'Triglycerides']

# Fill NaN values with the mode of the column for mode_columns
for col in mode_columns:
    heart_attack[col].fillna(heart_attack[col].mode()[0], inplace=True)

# Fill NaN values with the median of the column for median_columns
for col in median_columns:
    heart_attack[col].fillna(heart_attack[col].median(), inplace=True)

# Fill NaN values with the mean of the column for mean_columns
for col in mean_columns:
    heart_attack[col].fillna(heart_attack[col].mean(), inplace=True)

# heart_attack.fillna(-1e10, inplace=True)


unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)


# Check for NaN values in columns after fillna operations
columns_with_nulls = heart_attack.columns[heart_attack.isnull().any()].tolist()
print("Columns with NaN values after fillna operations:")
print(columns_with_nulls)

heart_attack.head()



[1 0]
[ 1. -1.]
[ 1. -1.]
[ 1.  0. -1.]
[0. 4. 3. 2. 1. 5.]
Columns with NaN values after fillna operations:
[]


,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,1,140,289,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,0
1,49,0,160,180,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,1
2,37,1,130,283,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,0
3,48,0,138,214,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,1
4,54,1,150,195,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,0


time: 16 ms (started: 2023-12-09 18:18:29 -05:00)


In [492]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")   # Use CPU
    print("GPU is not available, using CPU.")

GPU is not available, using CPU.
time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [493]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:23].values
Y = heart_attack.iloc[:, 24].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_val = scale.transform(X_val)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

X_train_t = torch.tensor(X_train, dtype=torch.float)
X_val_t = torch.tensor(X_val, dtype=torch.float)
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(X_train_t.dtype)
print(X_val_t.dtype)
print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [494]:
# def model_linear(t_in, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w20, w21, w22, w23, b):
#   t_p = ((w23 * t_in ** 23) + (w22 * t_in ** 22) + (w21 * t_in ** 21) +  (w20 * t_in ** 20) + (w19 * t_in ** 19) + (w19 * t_in ** 19) + 
#   (w18 * t_in ** 18) + (w17 * t_in ** 17) + (w16 * t_in ** 16) + (w15 * t_in ** 15) + (w14 * t_in ** 14) + (w13 * t_in ** 13) + 
#   (w12 * t_in ** 12) + (w11 * t_in ** 11) + (w10 * t_in ** 10) + (w9 * t_in ** 9) + (w8 * t_in ** 8) +
#   (w7 * t_in ** 7) + (w6 * t_in ** 6) + (w5 * t_in ** 5) + (w4 * t_in ** 4) + (w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
#   return t_p

# def loss_fn(t_p, t_gnd):
#   squared_diffs = (t_p - t_gnd)**2
#   return squared_diffs.mean()

# def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
#                     train_t_out, val_t_out):
#   for epoch in range(1, n_epochs + 1):

#     if params.grad is not None:
#       params.grad.zero_()

#     train_t_p = model_linear(train_t_in, *params)
#     train_loss = loss_fn(train_t_p, train_t_out)

#     val_t_p = model_linear(val_t_in, *params)
#     val_loss = loss_fn(val_t_p, val_t_out)

#     optimizer.zero_grad()
#     train_loss.backward()
#     optimizer.step()

#     if epoch <= 3 or epoch % 500 == 0:
#       print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
#       f" \tValidation loss {val_loss.item():.4f}")

#   return params

time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [495]:
# # params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)

# # learning_rate_1 = 1e-3# 
# optimizer_1 = optim.SGD([params], lr=learning_rate_1)
# # 
# print('Loss every 500 epochs using learning rate of 1e-14\n'# )
# training_loo# p(
#     n_epochs = 50# 00,
#     optimizer = optimize# r_1,
#     params = pa# rams,
#     train_t_in = X_tr# ain_t,
#     val_t_in = X# _val_t,
#     train_t_out = Y_# train_t,
#     val_t_out = Y_val_t)

time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [496]:
# # Calculate predictions for the validation set
# with torch.no_grad():
#     val_predictions = model_linear(X_val_t, *params)

# # Defining TP, TN, FP, FN
# TP = ((val_predictions == 1) & (Y_val_t == 1)).sum().item()
# TN = ((val_predictions == 0) & (Y_val_t == 0)).sum().item()
# FP = ((val_predictions == 1) & (Y_val_t == 0)).sum().item()
# FN = ((val_predictions == 0) & (Y_val_t == 1)).sum().item()

# print(X_train_t.shape)
# print(X_val_t.shape)
# print(val_predictions.shape)
# print(Y_val_t.shape)
# print(TP)
# print(TN)
# print(FP)
# print(FN)

time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [497]:
# # Defining PCA function
# def PCA_function(X_train, X_val, K):
#   pca = PCA(n_components=K)
#   X_train_PCA = pca.fit_transform(X_train)
#   X_val_PCA = pca.fit_transform(X_val)
#   return X_train_PCA, X_val_PCA

time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [498]:
#Sequential Model

model_seq = nn.Sequential(
            nn.Linear(23, 1024),
            nn.Tanh(),
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Linear(2048, 512),
            nn.Tanh(),
            nn.Linear(512,1)
            )
model_seq

Sequential(
  (0): Linear(in_features=23, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 16 ms (started: 2023-12-09 18:18:29 -05:00)


In [499]:
sum([p.numel() for p in model_seq.parameters()])

3173377

time: 16 ms (started: 2023-12-09 18:18:29 -05:00)


In [500]:
# # Finding the number of parameters (K) that results in the best accuracy

# K_vector = []
# recall_vector = []
# precision_vector = []
# accuracy_vector = []
# F1_score_vector = []

# for K in range(1, 24):
#   X_train_PCA, X_val_PCA = PCA_function(X_train, X_val, K)

#   classifier = LogisticRegression(random_state=0)
#   classifier.fit(X_train_PCA, Y_train)

#   Y_pred = classifier.predict(X_val_PCA)

#   K_vector.append(K)
#   recall_vector.append(metrics.recall_score(Y_val, Y_pred))
#   precision_vector.append(metrics.precision_score(Y_val, Y_pred))
#   accuracy_vector.append(metrics.accuracy_score(Y_val, Y_pred))
#   F1_score_vector.append(metrics.f1_score(Y_val, Y_pred))

#   if (metrics.accuracy_score(Y_val, Y_pred) > accuracy_at_K):
#     accuracy_at_K = metrics.accuracy_score(Y_val, Y_pred)
#     best_K = K

# print('Best K value: ', best_K)
# print('\nAccuracy using best K: ', accuracy_at_K)
# print('\nRecall history: ', recall_vector)
# print('\nAccuracy history: ', accuracy_vector)
# print('\nPrecision history: ', precision_vector)
# print('\nF1 score history: ', F1_score_vector)

time: 0 ns (started: 2023-12-09 18:18:29 -05:00)


In [501]:
#Let's define our training loop that will be used

def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 15 ms (started: 2023-12-09 18:18:29 -05:00)


In [502]:
#Let's try doing some training

optimizer_1 = optim.SGD(model_seq.parameters(), lr=1e-2)

training_loop(
    n_epochs = 51,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_t,
    x_val = X_val_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.3789, Validation loss: 0.3743
Epoch: 10, Training loss: 0.2556, Validation loss: 0.2531
Epoch: 20, Training loss: 0.2367, Validation loss: 0.2346
Epoch: 30, Training loss: 0.2342, Validation loss: 0.2322
Epoch: 40, Training loss: 0.2337, Validation loss: 0.2317
Epoch: 50, Training loss: 0.2336, Validation loss: 0.2316
time: 36.6 s (started: 2023-12-09 18:18:29 -05:00)


In [503]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positive = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positive) if (true_positives + false_positive) > 0 else 0

# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_t)))

# Calculate accuracy for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
print("Training Accuracy: {:.4f}".format(accuracy_train))

# Calculate accuracy for the training set
recall_train = calculate_recall(train_predictions, Y_train_t)
print("Training Recall: {:.4f}".format(recall_train))

# Calculate accuracy for the training set
precision_train = calculate_precision(train_predictions, Y_train_t)
print("Training Precision: {:.4f}".format(precision_train))

# Calculate accuracy for the training set
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)
print("Training F1 Score: {:.4f}".format(F1_score_train))


# Calculate accuracy for the validation set
accuracy_val = calculate_accuracy(train_predictions, Y_val_t)
print("Validation Accuracy: {:.4f}".format(accuracy_val))

# Calculate accuracy for the validation set
recall_val = calculate_recall(train_predictions, Y_val_t)
print("Validation Recall: {:.4f}".format(recall_val))

# Calculate accuracy for the validation set
precision_val = calculate_precision(train_predictions, Y_val_t)
print("Validation Precision: {:.4f}".format(precision_val))

# Calculate accuracy for the validation set
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.3825
Training Recall: 1.0000
Training Precision: 0.3825
Training F1 Score: 0.5533


RuntimeError: The size of tensor a (7987) must match the size of tensor b (1997) at non-singleton dimension 0

time: 250 ms (started: 2023-12-09 18:19:06 -05:00)


In [ ]:
model_fnn = nn.Sequential(
            nn.Linear(23, 1024),
            nn.Tanh(),
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 16 ms (started: 2023-12-09 17:43:43 -05:00)


In [ ]:
sum([p.numel() for p in model_fnn.parameters()])

4779649

time: 0 ns (started: 2023-12-09 17:43:44 -05:00)


In [ ]:
learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 51

#This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.696297
Epoch: 10, Loss: 0.659356
Epoch: 20, Loss: 0.658965
Epoch: 30, Loss: 0.658694
Epoch: 40, Loss: 0.658424
Epoch: 50, Loss: 0.658113
time: 46.4 s (started: 2023-12-09 17:30:02 -05:00)


In [ ]:
# Checking the Accuracy of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
print("Training Accuracy: %f" % accuracy_train)

precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
print("Training Precision: %f" % precision_train)

recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
print("Training Recall: %f" % recall_train)

F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.617503
Training Precision: 0.308752
Training Recall: 0.500000
Training F1 score: 0.381763
time: 328 ms (started: 2023-12-09 17:30:48 -05:00)


In [ ]:
# Checking the Accuracy of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
print("Validation Accuracy: %f" % accuracy_val)